In [1]:
from tensorflow.keras.applications import * #Efficient Net included here
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import os
import shutil
import pandas as pd
import numpy as np
from sklearn import model_selection
from tqdm import tqdm
from tensorflow.keras import optimizers
import tensorflow as tf

Using TensorFlow backend.


In [2]:
width = 150
height = 150
input_shape = (height, width, 3)
conv_base = EfficientNetB6(weights="imagenet", include_top=False, input_shape=input_shape)

In [3]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
#avoid overfitting
model.add(layers.Dropout(rate=0.2, name="dropout_out"))
conv_base.trainable = False

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb6 (Functional)  (None, 5, 5, 2304)        40960143  
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 2304)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 2304)              0         
Total params: 40,960,143
Trainable params: 0
Non-trainable params: 40,960,143
_________________________________________________________________


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [6]:
def image_to_vector(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    return result